In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install tensorflow-gpu keras tensorflow-hub

In [30]:
cd /content/drive/My\ Drive/Clothes-Classification

/content/drive/My Drive/Clothes-Classification


In [7]:
cd zips

/content/drive/.shortcut-targets-by-id/1CXf90C7MJtFG2QnRPpFhGu3hLraVH-pi/DATASET SIH/zips


In [8]:
!unrar x -r "*.rar"
!unzip "*.zip"

Streaming output truncated to the last 5000 lines.
  inflating: skirt/images193.jpg     
  inflating: skirt/images194.jpg     
  inflating: skirt/images195.jpg     
  inflating: skirt/images196.jpg     
  inflating: skirt/images197.jpg     
  inflating: skirt/images198.jpg     
  inflating: skirt/images1hjk.jpg    
  inflating: skirt/images1rty.jpg    
  inflating: skirt/images2.jpg       
  inflating: skirt/images200.jpg     
  inflating: skirt/images201.jpg     
  inflating: skirt/images202.jpg     
  inflating: skirt/images203.jpg     
  inflating: skirt/images204.jpg     
  inflating: skirt/images205.jpg     
  inflating: skirt/images206.jpg     
  inflating: skirt/images207.jpg     
  inflating: skirt/images208.jpg     
  inflating: skirt/images209.jpg     
  inflating: skirt/images21.jpg      
  inflating: skirt/images210.jpg     
  inflating: skirt/images211.jpg     
  inflating: skirt/images212.jpg     
  inflating: skirt/images213.jpg     
  inflating: skirt/images214.jpg     

In [9]:
% cd ../
!rm -rf train/ test/ temp/

/content/drive/.shortcut-targets-by-id/1CXf90C7MJtFG2QnRPpFhGu3hLraVH-pi/DATASET SIH


In [0]:
!mkdir train
!mkdir test

In [31]:
!ls 

 clothes  'Copy of model.ipynb'   data	 model.h5   model.json


In [12]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

import matplotlib.pyplot as plt
import matplotlib.style as style
from datetime import datetime
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
from tensorflow.keras import layers
from time import time
import os
import pandas
import shutil
import numpy as np
print("TF version:", tf.__version__)

TF version: 2.2.0


In [13]:
classes = sorted([x for x in os.listdir("zips") if os.path.isdir(os.path.join("zips", x))])
operation = ['train', 'test']
print(classes)

['Blazer', 'Burkha', 'Headwear', 'Long pants', 'Scarf', 'Sweater', 'Vest', 'bags', 'chudidar', 'hoddie', 'jeans', 'jersey', 'kurta', 'saree', 'shirt', 'shoes', 'skirt', 'strip-dress', 'sunglasses', 'tops', 'trousers', 'tshirt']


In [0]:
def convert(path, image):
  if os.path.splitext(image)[1] == "gif":
    print(image, path)
    os.remove(os.path.join(path, image))
    return 
  else:
    try:
      im = Image.open(os.path.join(path, image))
      rgb_im = im.convert('RGB')
      rgb_im.load()
    except:
      os.remove(os.path.join(path, image))
  

In [0]:
for id in classes:
  for op in operation:
    if not os.path.exists(os.path.join(op, id)):
      os.mkdir(os.path.join(op, id))

In [0]:
train_dest = os.path.join('train')
test_dest = os.path.join('test')
for label in classes:
  origin = os.path.join('zips', label)
  data = os.listdir(origin)
  train = data[:int(np.ceil(0.8*len(data)))]
  test =  data[int(np.ceil(0.8*len(data))):]
  for images in train:
    shutil.move(os.path.join(origin, images), os.path.join(train_dest, label,  images))
  for images in test:
    shutil.move(os.path.join(origin, images), os.path.join(test_dest, label,  images))


In [17]:
from termcolor import colored
import warnings
for id in classes:
  for op in operation:
    for image in os.listdir(os.path.join(op, id)):
      convert(os.path.join(op, id), image)


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [18]:
pixels = 224
FV_SIZE = 1280
IMAGE_SIZE = (pixels, pixels)
BATCH_SIZE = 64
model = tf.keras.Sequential([
  hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", 
                  input_shape=IMAGE_SIZE+(3,),
                 output_shape=[1280],
                 trainable=False),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(rate=0.2),
  tf.keras.layers.Dense(len(classes), activation='softmax')
])                                   
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               655872    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 22)                11286     
Total params: 2,925,142
Trainable params: 667,158
Non-trainable params: 2,257,984
_________________________________________________________________


In [19]:
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    test_dest, 
    shuffle=False, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

train_datagen = validation_datagen
  
train_generator = train_datagen.flow_from_directory(
    train_dest, 
    subset="training", 
    shuffle=True, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

Found 3275 images belonging to 22 classes.
Found 13148 images belonging to 22 classes.


In [0]:
LEARNING_RATE = 0.001
model.compile(
   optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE), 
   loss='categorical_crossentropy',
   metrics=['accuracy'])

In [21]:
EPOCHS=10
STEPS_EPOCHS = train_generator.samples//train_generator.batch_size
VALID_STEPS=validation_generator.samples//validation_generator.batch_size
history = model.fit( 
          train_generator,
          steps_per_epoch=STEPS_EPOCHS,
          epochs=EPOCHS,
          validation_data=validation_generator,
          validation_steps=VALID_STEPS)

Epoch 1/10
  4/205 [..............................] - ETA: 45s - loss: 3.1413 - accuracy: 0.1680

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


205/205 [==============================] - 106s 518ms/step - loss: 0.8452 - accuracy: 0.7345 - val_loss: 0.6077 - val_accuracy: 0.7834
Epoch 2/10
205/205 [==============================] - 104s 507ms/step - loss: 0.4920 - accuracy: 0.8312 - val_loss: 0.6010 - val_accuracy: 0.7883
Epoch 3/10
205/205 [==============================] - 104s 509ms/step - loss: 0.4232 - accuracy: 0.8517 - val_loss: 0.5131 - val_accuracy: 0.8162
Epoch 4/10
205/205 [==============================] - 104s 506ms/step - loss: 0.3772 - accuracy: 0.8669 - val_loss: 0.5508 - val_accuracy: 0.8091
Epoch 5/10
205/205 [==============================] - 103s 504ms/step - loss: 0.3269 - accuracy: 0.8877 - val_loss: 0.5069 - val_accuracy: 0.8208
Epoch 6/10
205/205 [==============================] - 102s 496ms/step - loss: 0.3023 - accuracy: 0.8928 - val_loss: 0.5248 - val_accuracy: 0.8208
Epoch 7/10
205/205 [==============================] - 103s 501ms/step - loss: 0.2782 - accuracy: 0.9016 - val_loss: 0.5212 - val_accura

In [0]:
model_json = model.to_json()
path = "/content/drive/My Drive/Clothes-Classification"
with open(os.path.join(path,"model.json"), "w") as json_file:
    json_file.write(model_json)
tf.keras.models.save_model(model, os.path.join(path,'model.h5'))

In [35]:
ls

 clothes@  'Copy of model.ipynb'   data/   model.h5   model.json


In [32]:
import itertools
import random
from collections import Counter
from glob import iglob
import cv2
def load_image(filename):
    img = cv2.imread(filename)
    img = cv2.resize(img, (IMAGE_SIZE[0], IMAGE_SIZE[1]) )
    img = img /255
    
    return img
def predict(image):
    probabilities = model.predict(np.asarray([img]))[0]
    class_idx = np.argmax(probabilities)
    
    return {classes[class_idx]: probabilities[class_idx]}
for idx, filename in enumerate(random.sample(validation_generator.filenames, 5)):
    print("SOURCE: class: %s, file: %s" % (os.path.split(filename)[0], filename))
    img = load_image(os.path.join('test' , filename))
    prediction = predict(img)
    print("PREDICTED: class: %s, confidence: %f" % (list(prediction.keys())[0], list(prediction.values())[0]))
    plt.imshow(img)
    plt.figure(idx)    
    plt.show()
    
img = load_image('woman.jpg')
prediction = predict(img)
print("PREDICTED: class: %s, confidence: %f" % (list(prediction.keys())[0], list(prediction.values())[0]))
plt.imshow(img)
plt.figure(idx)    
plt.show()

SOURCE: class: shirt, file: shirt/Onerio-100-Percent-Cotton-Peach-SDL142426370-2-b9953.jpg


error: ignored

In [0]:
ls

'Copy of model.ipynb'   model.h5     pants_linen.png
 data/                  model.json   woman.jpg
